In [ ]:
'''
token = "0q4iBYUKXGyoaQkhb95o3I8Lh4_-JoT-uyoCs5Hg7LuN"
service = QiskitRuntimeService.save_account(
channel="ibm_cloud",
instance="Free Plan",
token=token,
set_as_default=True,
overwrite=True
)
'''

In [1]:
# PennyLane imports
import pennylane as qml

from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton

# General imports
import os
import json
import numpy as np
from datetime import datetime, timedelta

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian

from qiskit_ibm_runtime import QiskitRuntimeService, Session


In [2]:
service = QiskitRuntimeService()
backend = service.backend('ibm_brisbane')

In [3]:
potential = 'AHO'
cutoff = 4
shots = 2

In [4]:
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])[:4]
min_eigenvalue = min(eigenvalues.real)

num_qubits = int(1 + np.log2(cutoff))

In [13]:
dev = qml.device('qiskit', wires=num_qubits, backend=backend, shots=shots)

@qml.qnode(dev)
def cost_function(params):

    basis_state = [1] + [0]*(num_qubits-1)
    qml.BasisState(basis_state, wires=range(num_qubits))

    qml.RY(params[0], wires=1)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))
    

DeviceError: Device qiskit does not exist. Make sure the required plugin is installed.

In [8]:
params = np.random.rand(1)
cost_function(params)

c:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\.venv\Lib\site-packages\pennylane_qiskit\qiskit_device.py:224: UserWarning: The observable measured Hermitian(array([[ 3.4375    +0.j,  0.        +0.j,  6.09879599+0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j],
       [ 0.        +0.j, 13.0625    +0.j,  0.        +0.j,
        10.56342452+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j],
       [ 6.09879599+0.j,  0.        +0.j, 22.6875    +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j],
       [ 0.        +0.j, 10.56342452+0.j,  0.        +0.j,
        13.0625    +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.9375    +0.j,  0.        +0.j,
         3.97747564+0.j,  0.        +0.j],
       [ 0.    

RequestsApiError: '400 Client Error: Bad Request for url: https://us-east.quantum-computing.cloud.ibm.com/sessions. {"errors":[{"code":1352,"message":"You are not authorized to run a session when using the open plan.","solution":"Create an instance of a different plan type or use a different execution mode <https://quantum.cloud.ibm.com/docs/guides/execution-modes>.","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"ac117d02-6d4a-49cf-9ad2-3f32bb109c9d"}\n'

In [6]:
# VQE
vqe_start = datetime.now()

#variables
num_vqe_runs = 1
max_iter = 100
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 5

# Generate Halton sequence
num_dimensions = 1
num_samples = popsize
halton_sampler = Halton(d=num_dimensions)
halton_samples = halton_sampler.random(n=num_samples)
scaled_samples = 2 * np.pi * halton_samples

#data arrays
energies = []
x_values = []
success = []
run_times = []
num_iters = []
num_evaluations = []

#Optimizer
bounds = [(0, 2 * np.pi) for _ in range(num_dimensions)]


for i in range(num_vqe_runs):

    run_start = datetime.now()

    if i % 1 == 0:
        print(f"Run: {i}")

    # Differential Evolution optimization
    res = differential_evolution(cost_function,
                                bounds,
                                maxiter=max_iter,
                                tol=tol,
                                atol=abs_tol,
                                strategy=strategy,
                                popsize=popsize,
                                init=scaled_samples
                                )
    
    if res.success == False:
        print("Not converged")

    energies.append(res.fun)
    x_values.append(res.x)
    success.append(res.success)
    num_iters.append(res.nit)
    num_evaluations.append(res.nfev)

    run_end = datetime.now()
    run_time = run_end - run_start
    run_times.append(run_time)

total_run_time = sum(run_times, timedelta())
vqe_end = datetime.now()
vqe_time = vqe_end - vqe_start

Run: 0


c:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\.venv\Lib\site-packages\pennylane_qiskit\qiskit_device.py:224: UserWarning: The observable measured Hermitian(array([[ 3.4375    +0.j,  0.        +0.j,  6.09879599+0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j],
       [ 0.        +0.j, 13.0625    +0.j,  0.        +0.j,
        10.56342452+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j],
       [ 6.09879599+0.j,  0.        +0.j, 22.6875    +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j],
       [ 0.        +0.j, 10.56342452+0.j,  0.        +0.j,
        13.0625    +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.9375    +0.j,  0.        +0.j,
         3.97747564+0.j,  0.        +0.j],
       [ 0.    

RequestsApiError: '400 Client Error: Bad Request for url: https://us-east.quantum-computing.cloud.ibm.com/sessions. {"errors":[{"code":1352,"message":"You are not authorized to run a session when using the open plan.","solution":"Create an instance of a different plan type or use a different execution mode <https://quantum.cloud.ibm.com/docs/guides/execution-modes>.","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"57926335-2707-421f-9374-270a4219a56c"}\n'

In [ ]:
res

In [ ]:
# Save run
run = {
    "starttime": str(vqe_start),
    "potential": potential,
    "cutoff": cut_off,
    "exact_eigenvalues": [x.real.tolist() for x in eigenvalues],
    "ansatz": "StronglyEntanglingLayers-1layer",
    "num_VQE": num_vqe_runs,
    "shots": shots,
    "Optimizer": {
        "name": "differential_evolution",
        "bounds": "[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]",
        "maxiter": max_iter,
        "tolerance": tol,
        "abs_tolerance": abs_tol,
        "strategy": strategy,
        "popsize": popsize,
        'init': 'scaled_samples',
    },
    "results": energies,
    "params": [x.tolist() for x in x_values],
    "num_iters": num_iters,
    "num_evaluations": num_evaluations,
    "success": np.array(success, dtype=bool).tolist(),
    "run_times": [str(x) for x in run_times],
    "parallel_run_time": str(vqe_time),
    "total_VQE_time": str(total_run_time),
    "total_device_time": str(device_time),
    "Backend": backend.name
}

run

In [64]:
# Save the variable to a JSON file
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQE\Differential Evolution\Real Device\Files\\"
path = base_path + "{}_{}.json".format(potential, cut_off)
with open(path, 'w') as json_file:
    json.dump(run, json_file, indent=4)